# 🎓 Gaussian Fitting and Peak Detection
Welcome to this interactive notebook for exploring Gaussian fitting and peak detection.

In this notebook, you will:
- Upload your own data file (.csv or .xls)
- Select which columns represent X and Y
- Detect peaks in your data
- Fit your data to a sum of Gaussian curves
- Evaluate fit quality using statistics
- Export results and plots

Let's get started!

In [ ]:
# 📦 Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from ipywidgets import interact, widgets, fixed
from IPython.display import display, clear_output
from datetime import datetime
import io
from google.colab import files

## 📁 Upload Your Data File
Use the widget below to upload a `.csv` or `.xls` file containing your data.

In [ ]:
# Upload file
uploaded = files.upload()

# Load the file into a DataFrame
for fname in uploaded:
    if fname.endswith('.csv'):
        df = pd.read_csv(io.BytesIO(uploaded[fname]))
    elif fname.endswith('.xls') or fname.endswith('.xlsx'):
        df = pd.read_excel(io.BytesIO(uploaded[fname]))
    else:
        raise ValueError("Unsupported file format. Please upload a .csv or .xls file.")

print("✅ File loaded successfully!")
df.head()

## 📊 Select X and Y Columns
Choose which columns in your data represent the X and Y values for analysis.

In [ ]:
# Create dropdowns for column selection
x_dropdown = widgets.Dropdown(options=df.columns.tolist(), description='X Column:')
y_dropdown = widgets.Dropdown(options=df.columns.tolist(), description='Y Column:')
display(x_dropdown, y_dropdown)

## 🔍 Peak Detection
Detect peaks in your Y data and visualize them on a plot.

In [ ]:
# Detect and plot peaks
def detect_and_plot_peaks(x_col, y_col):
    x = df[x_col].values
    y = df[y_col].values
    peaks, _ = find_peaks(y, height=np.mean(y))
    plt.figure(figsize=(10, 4))
    plt.plot(x, y, label='Data')
    plt.plot(x[peaks], y[peaks], 'ro', label='Detected Peaks')
    plt.title('Peak Detection')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.grid(True)
    plt.show()
    return peaks

peaks = detect_and_plot_peaks(x_dropdown.value, y_dropdown.value)

## ✅ Accept or Override Peak Guesses
Choose whether to use the detected peaks or manually input initial guesses.

In [ ]:
# Toggle for accepting peak guesses
accept_peaks = widgets.ToggleButtons(
    options=['Accept Detected Peaks', 'Input Manually'],
    description='Peak Init:',
    style={'description_width': 'initial'}
)
display(accept_peaks)

## ⚙️ Baseline Toggle
Choose whether to include a baseline in the Gaussian fit.

In [ ]:
# Checkbox for baseline fitting
fit_baseline = widgets.Checkbox(value=True, description='Fit Baseline')
display(fit_baseline)

## 📈 Fit Gaussians to Data
Fit the data to a sum of Gaussians using the selected parameters.

In [ ]:
# Define Gaussian model
def gaussian(x, A, mu, sigma):
    return A * np.exp(-((x - mu)**2) / (2 * sigma**2))

def multi_gaussian(x, *params):
    n = (len(params) - 1) // 3 if fit_baseline.value else len(params) // 3
    y = np.zeros_like(x)
    for i in range(n):
        A = params[3*i]
        mu = params[3*i+1]
        sigma = params[3*i+2]
        y += gaussian(x, A, mu, sigma)
    if fit_baseline.value:
        y += params[-1]
    return y

# Prepare initial guesses
x = df[x_dropdown.value].values
y = df[y_dropdown.value].values
if accept_peaks.value == 'Accept Detected Peaks':
    init_params = []
    for p in peaks:
        init_params += [y[p], x[p], 0.1]
    if fit_baseline.value:
        init_params += [np.min(y)]
else:
    num_gaussians = int(input("Enter number of Gaussians: "))
    init_params = []
    for i in range(num_gaussians):
        A = float(input(f"Amplitude {i+1}: "))
        mu = float(input(f"Mean {i+1}: "))
        sigma = float(input(f"Sigma {i+1}: "))
        init_params += [A, mu, sigma]
    if fit_baseline.value:
        init_params += [np.min(y)]

# Fit the model
popt, pcov = curve_fit(multi_gaussian, x, y, p0=init_params)
y_fit = multi_gaussian(x, *popt)

# Plot fit
plt.figure(figsize=(10, 4))
plt.plot(x, y, label='Data')
plt.plot(x, y_fit, 'r--', label='Gaussian Fit')
plt.title('Gaussian Fit')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True)
plt.show()

## 📊 Fit Statistics
Evaluate the quality of the fit using R², RMSE, and AIC.

In [ ]:
# Compute fit statistics
residuals = y - y_fit
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
rmse = np.sqrt(ss_res / len(y))
aic = len(y) * np.log(ss_res / len(y)) + 2 * len(popt)

print(f"R²: {r_squared:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"AIC: {aic:.2f}")

## 💾 Export Results
Save the fitted data and plot with a timestamped filename.

In [ ]:
# Export results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename_csv = f"gaussian_fit_{timestamp}.csv"
filename_png = f"gaussian_plot_{timestamp}.png"

# Save data
result_df = pd.DataFrame({'X': x, 'Y': y, 'Y_Fit': y_fit})
result_df.to_csv(filename_csv, index=False)

# Save plot
plt.figure(figsize=(10, 4))
plt.plot(x, y, label='Data')
plt.plot(x, y_fit, 'r--', label='Gaussian Fit')
plt.title('Gaussian Fit')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True)
plt.savefig(filename_png)
plt.close()

print(f"✅ Results saved as {filename_csv} and {filename_png}")